<a href="https://colab.research.google.com/github/GiuliaLanzillotta/exercises/blob/master/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment analysis of tweets
This notebook has been used in the *Kaggle Competition* https://www.kaggle.com/c/cil-text-classification-2020/data.

---

### The pipeline 
1. Train the embeddings to obtain a representation of each word
2. Compose the word embeddings to get a representation of each tweet
3. Train a classifier to distinguish between positive and negative tweets

## Data gathering 
The dataset used comes from Twitter. 
<br>
*Note: I have temporarly stored a copy  of the .zip file on my Drive to be able to access the data from here. <br>The following lines access my Drive folder.*

In [1]:
!unzip 'drive/My Drive/cil-text-classification-2020.zip'

Archive:  drive/My Drive/cil-text-classification-2020.zip
  inflating: README.md               
  inflating: build_vocab.sh          
  inflating: cooc.py                 
  inflating: cut_vocab.sh            
  inflating: pickle_vocab.py         
  inflating: sample_submission.csv   
  inflating: test_data.txt           


In [2]:
!unzip 'drive/My Drive/twitter-datasets.zip'

Archive:  drive/My Drive/twitter-datasets.zip
  inflating: twitter-datasets/sample_submission.csv  
  inflating: twitter-datasets/test_data.txt  
  inflating: twitter-datasets/train_neg_full.txt  
  inflating: twitter-datasets/train_neg.txt  
  inflating: twitter-datasets/train_pos_full.txt  
  inflating: twitter-datasets/train_pos.txt  


In [3]:
!ls

build_vocab.sh	drive		 sample_data		twitter-datasets
cooc.py		pickle_vocab.py  sample_submission.csv
cut_vocab.sh	README.md	 test_data.txt


In [0]:
positive_location = 'twitter-datasets/train_pos.txt'
negative_location = 'twitter-datasets/train_neg.txt'

In [6]:
!head -3 'twitter-datasets/train_pos.txt'

<user> i dunno justin read my mention or not . only justin and god knows about that , but i hope you will follow me #believe 15
because your logic is so dumb , i won't even crop out your name or your photo . tsk . <url>
" <user> just put casper in a box ! " looved the battle ! #crakkbitch


In [8]:
!wc -l 'twitter-datasets/train_pos.txt'
!wc -l 'twitter-datasets/train_pos_full.txt'

100000 twitter-datasets/train_pos.txt
1250000 twitter-datasets/train_pos_full.txt


Okay so our dataset consists of:<br>
**10.000** entries for the little file<br>
**1.250.000** entries for the full file -**x2** because we have a positive and negative file<br>
<br>

## Embeddings 

### Co-occurrence matrix 

Since the dataset is this big we directly build the co-occurrence matrix from the text file, without loading it into a Python structure

In [47]:
# To build a co-occurrence matrix we have to do the following steps:
# 1. build a vocabulary of words - you can work on this vocabulary as a pre-processing step of the pipeline
# 2. match the words in each line to the vocabulary and build a co-occurence matrix based on a notion of window

# Here we build the vocabulary text
!cat 'twitter-datasets/train_pos.txt' 'twitter-datasets/train_neg.txt'  | sed "s/ /\n/g" | grep -v "^\s*$" | sort | uniq > vocab.txt
!wc -l './vocab.txt'

114427 ./vocab.txt


In [0]:
# Now let's load the vocabulary into a Python dictionary and serialise it 
# To serialise a Python object we use the library pickle
import pickle

vocab = dict()
with open('vocab.txt') as f:
  for idx, line in enumerate(f):
      vocab[line.strip()] = idx

with open('vocab.pkl', 'wb') as f:
  pickle.dump(vocab, f, pickle.HIGHEST_PROTOCOL)

In [49]:
!ls

build_vocab.sh	drive		 sample_data		twitter-datasets
cooc.py		pickle_vocab.py  sample_submission.csv	vocab.pkl
cut_vocab.sh	README.md	 test_data.txt		vocab.txt


Now it is the time to build the **co-occurrence matrix**. <br>
In this step we make a few important design choices, namely :
- We define the size of the *co-occurrence window*, which implicitly defines our concept of *context*
- We define a *weighting scheme*, which has implications on the type of meaning we are trying to medl (a more syntactical one, instead of a more semantical one) 
<br>


```
# Several choices can (and should be) be tried out here
# I would make a distinction between two main kind of co-occurrence matrices: 
# the first one uses a large window (we can use the whole line) and no weighting, 
# while the second one uses a small window with a distance weighting. 

semantic_matrix = ... # large window, no weighting
syntactic matrix = ... # small window, 1/n weighting
```



In [0]:
from scipy.sparse import *
import numpy as np

# Now let's define an helper function to build a co-occurrence matrix 
def build_cooc(vocab_location, original_files, 
               window_size=None, weighting="None", output_location="./cooc.pkl"):
  """ 
  Parameters: 
  - vocab_location: a path to the .pkl file containing the vocabulary 
  - original_files: a list containing the paths to the input files
  - window_size : if None, using the whole line as a window, otw a number is expected. 
    Note: the window size cannot be larger tha the line length.
  - weighting: only 2 types supported for now, one of 'None' or 'Distance' 
  - output_location: a path to the .pkl file containing the co-occurrence matrix

  By default the output (co-occurence matrix) will be saved in a .pkl file in the current
  working directory.
  """
  # load the vocabulary 
  with open(vocab_location, 'rb') as f:
        vocab = pickle.load(f)
  vocab_size = len(vocab)


  data, row, col = [], [], []
  counter = 1
  # opening each file
  for fn in original_files:
      with open(fn) as f:
        print("Working on ",fn)
        # looking at each line
        for line in f:
          # Here we filter out the words that are not in the vocabulary 
          tokens = [vocab.get(t, -1) for t in line.strip().split()]
          tokens = [t for t in tokens if t >= 0]
          ll = len(tokens) # filtered line length
          if window_size==None or window_size>=ll:
            delta = ll 
          else: 
            delta = window_size 
          
          for j in range(ll):
              t1 = tokens[j]
              for i in range(-1*delta,delta):
                if j+i<0 or j+i>=ll or i==0:
                  # Note: I exclude the self-co-occurrence 
                  # to save space in memory 
                  continue
                t2 = tokens[j+i]
                c = 1
                if weighting == 'Distance':
                  c = c/i
                data.append(c)
                row.append(t1)
                col.append(t2)

          if counter % 10000 == 0:
              print(counter)
          counter += 1
            
  # According to scipy documentation the duplicate indices 
  # entries are not summed automatically
  cooc = coo_matrix((data, (row, col)))
  print("summing duplicates (this can take a while)")
  cooc.sum_duplicates()

  # Saving the output
  with open(output_location, 'wb') as f:
      pickle.dump(cooc, f, pickle.HIGHEST_PROTOCOL)


In [0]:
file_locations = [positive_location, negative_location]
# First co-occurrence matrix 
build_cooc('./vocab.pkl', file_locations, output_location='./first_cooc.pkl')

In [0]:
# Second co-occurrence matrix 
build_cooc('./vocab.pkl',file_locations, window_size=4, weighting='Distance', output_location='./second_cooc.pkl')

In [6]:
# Saving to Google Drive 
!cp vocab.pkl 'drive/My Drive/'
!cp first_cooc.pkl 'drive/My Drive/' 
!cp second_cooc.pkl 'drive/My Drive/' 

"\n!cp first_cooc.pkl 'drive/My Drive/' \n!cp second_cooc.pkl 'drive/My Drive/' "

### GloVe training
The GloVe objective is a **weighted least squares fit of log-counts**.<br>
We'll train two word embeddings ($X$, $Y$) with stochastic gradient descent to maximise the GloVe objective.


In [109]:
# First we load the co-occurrence matrix we are training on 
cooc_location = 'first_cooc.pkl' # semantic cooc
print("loading cooccurrence matrix")
with open(cooc_location, 'rb') as f:
    cooc = pickle.load(f)
print("{} nonzero entries".format(cooc.nnz))

loading cooccurrence matrix
10121875 nonzero entries


Here we have to define some **GloVe hyperparameters**. <br>This assignments should not come out of a rule-of-thumb, rather they should be based on the data as much as possible. 

In [110]:
# Now let's use a simple reasoning here. 
# Let's define  beta  as the frequency treshold, meaning that a frequent 
# pair of word should have a frequency  > beta.

# Let's say a frequent word pair should appear in at least 5% of the tweets
beta = 0.05 

# We are working with 10.000 tweets for now
ntweets = 10000

# The hyperparameter MAX is then defined as : 
MAX = beta*ntweets
print("MAX: ",MAX)

# Other hyperparameters
ALPHA = 3/4 # this is the discount factor to apply to the frequent words
print("ALPHA: ", ALPHA)
EMBEDDING_DIM = 100 # the number of dimensions has lots of implications, and 
print("EMBEDDING DIM: ", EMBEDDING_DIM)
#there's no reasoning that can justify the right number 

MAX:  500.0
ALPHA:  0.75
EMBEDDING DIM:  100


In [0]:
# Initialise the embeddings
xs = np.random.normal(size=(cooc.shape[0], EMBEDDING_DIM))
ys = np.random.normal(size=(cooc.shape[1], EMBEDDING_DIM))

Here we implement **SGD** and we define some of its hyperparameters.

---
```
#TODO : modify Glove objective to take into consideration different similarity
# metrics between words (the standard one relies on the dot product)
```
---


In [101]:
eta = 0.0001
epochs = 10
embedding_locations = 'embeddings1'

for epoch in range(epochs):
  print("epoch {}".format(epoch))
  for ix, jy, n in zip(cooc.row, cooc.col, cooc.data):
    logn = np.log(n)
    fn = min(1.0, (n / MAX) ** ALPHA)
    x, y = xs[ix, :], ys[jy, :]
    scale = 2 * eta * fn * (logn - np.dot(x, y))
    xs[ix, :] += scale * y
    ys[jy, :] += scale * x
np.savez(embedding_locations, xs, ys)

epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9


In [0]:
!cp 'embeddings1.npz' 'drive/My Drive/'

Now that we have finished training the embeddings we want to have **a quick overview of what is the result**. <br>
To be able to do that we first need to integrate our vocabulary with the embedding matrix.

In [111]:
# Re-load the embeddings
npzfile = np.load('embeddings1.npz')
npzfile.files

['arr_0', 'arr_1']

In [0]:
# Load the vocabulary to extract the association index -> word 
with open('vocab.pkl', 'rb') as f:
      vocab = pickle.load(f)

In [0]:
# Transfer the embeddings into a dataframe 
import pandas as pd
# Now we need to substitute the indexes with the corresponding words, 
# as given in the vocabulary 
keys = vocab.keys()
X_df = pd.DataFrame(npzfile['arr_0'], index=keys)
Y_df = pd.DataFrame(npzfile['arr_1'], index=keys)

In [127]:
X_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
`,0.473936,-2.176287,-0.780994,-0.582022,1.096394,1.439894,1.213193,1.036713,0.507574,0.669754,-0.416431,0.205280,-1.505057,0.165941,0.520289,1.588283,-0.828317,-0.605844,0.107087,0.837186,-0.314274,-0.759787,-0.249638,-0.624133,1.134519,0.966436,0.642990,-1.702596,1.138297,-0.045424,-0.758173,0.265954,0.020887,1.687514,-0.771104,0.670245,0.110943,0.675248,-0.683348,-1.011031,...,-0.877079,0.289534,0.412858,-0.744307,0.413928,-1.624588,2.281203,0.015445,-1.730453,0.591234,-1.122728,1.195836,-1.553392,-0.054038,-0.664035,0.914970,-1.013892,-0.941261,-0.320293,1.809805,1.498340,1.463693,-0.840383,-0.474247,0.803659,-0.699898,0.426789,-1.616629,-0.934522,-2.384379,0.928403,-0.314667,0.480708,0.031845,1.227262,0.018915,-0.359059,1.110359,-0.564217,-0.118700
^,0.289559,-1.868453,1.134751,0.502029,-1.257387,-1.146697,0.867599,0.929203,0.062321,-0.869012,0.513309,-0.585224,-0.994814,0.099877,0.650158,-0.668259,0.807061,0.116481,0.980497,0.619730,0.640612,-0.915360,0.649427,0.134223,-0.565159,1.135901,-1.197937,1.969857,0.317963,-0.638805,-0.861153,0.211646,-1.069356,0.934066,-0.860682,1.579404,0.656490,-0.765495,0.464613,1.104234,...,0.910024,0.688463,-0.022465,-0.088749,0.247602,-0.437997,0.902696,-0.418634,0.081079,0.300992,1.104750,0.599431,-0.269272,0.604846,-0.362225,-0.216814,-0.090024,1.908220,0.208957,0.392682,-0.627514,1.258340,-1.698121,-0.219329,-0.373388,-0.758205,-1.151123,-0.398018,-0.009797,1.831295,-0.498855,0.699288,0.614120,0.055377,-0.855412,-0.760369,-1.444465,-2.255991,-0.154119,-0.492416
~,1.178408,-0.672965,-0.497189,-0.810477,-0.895652,0.110635,-1.042897,-0.710021,0.074419,0.777404,-1.324407,0.452558,-0.207527,-0.090533,0.661557,1.362989,0.394902,-0.674951,-0.600683,1.871310,-1.050641,-0.072039,-0.413957,0.577257,-0.914736,-0.989615,-1.239333,-1.165745,0.995368,-1.944461,-0.439282,-1.657235,0.236989,0.857301,-0.006548,-0.234861,0.722585,-0.799353,-0.167051,-0.540751,...,0.782462,1.443861,-0.886983,0.567317,-0.125966,-0.259720,-0.497193,0.811501,0.616308,0.583513,-0.440980,-1.857846,0.596675,-1.422876,-0.314646,-1.718516,0.000130,1.321182,-0.879390,0.832289,-0.794130,-0.437417,0.658336,-1.995509,-0.197276,0.399674,1.666053,-1.372860,1.109184,-0.050761,-1.122224,0.166420,-1.211621,1.321194,-0.222103,0.559735,0.213144,0.973547,0.220861,1.067820
×,-1.301938,-0.221869,-1.339770,-1.289122,0.351518,1.447174,-0.461569,1.964100,1.508901,-0.434748,-0.422564,-2.072223,0.034195,1.665451,-1.118646,1.426851,0.353533,-0.272413,-1.193303,0.113430,0.445754,-0.089027,-0.560299,-0.760512,-0.287211,0.734672,0.507651,-0.337162,-0.545904,-0.293745,1.277089,-0.379277,-0.620461,0.811449,1.470436,2.191121,-0.658830,0.864191,0.442519,-0.624450,...,0.169492,-0.855915,0.324772,0.530432,0.417808,-0.744394,1.148376,-0.970921,-2.058514,-0.675513,0.300095,-0.581709,0.534720,-1.116369,0.414033,0.418269,-0.735615,0.933798,1.361246,1.237684,-1.152001,0.092433,2.471461,-0.344984,0.411856,1.450843,-0.797478,-1.097178,0.180615,-0.432896,-0.896195,0.612991,-0.588402,-0.310230,1.227507,-0.296647,-1.146949,2.776162,-0.369591,1.293035
<,1.255780,0.268804,0.770104,-0.447349,1.008445,0.181766,-0.048524,-0.251425,-1.733353,0.628797,1.270918,-0.055002,0.012452,-1.005026,0.243341,1.691085,0.231184,-1.200860,0.163073,0.626229,0.070585,0.143458,-0.414321,-0.128827,-0.877579,0.713582,-0.127578,-0.214490,0.019982,-0.103662,0.297082,-0.294494,0.226625,0.733855,0.824954,-1.148233,-2.012519,1.004830,-0.473306,-0.692251,...,0.285825,1.763377,-0.674509,-0.263090,-0.267436,-1.987921,1.763837,-0.314442,0.642990,0.061467,0.640279,0.222577,-0.519906,-0.452409,-0.101849,0.593100,-0.537599,0.319871,1.779960,0.323549,-0.401870,0.206373,0.635499,0.305064,-0.721673,0.899752,0.832585,0.408215,0.224310,-0.487612,-1.157295,-1.112730,0.006064,0.58

#### VSM utils
*Note*: The following code is inspired by the ```Stanford University VSM package```, which is available at the goihub repository https://github.com/cgpotts/cs224u.



In [0]:
import scipy
import scipy.spatial.distance

# We here define a function that, given a measure of similarity between words, 
# and the embedding, computes the "neighbors" of an input word
def neighbors(word, df, distfunc, df2=None):
    """Tool for finding the nearest neighbors of `word` in `df` according
    to `distfunc`. The comparisons are between row vectors.
    Parameters
    ----------
    word : str
        The anchor word. Assumed to be in `rownames`.
    df : pd.DataFrame
        The vector-space model.
    distfunc : function mapping vector pairs to floats.
        The measure of distance between vectors. Can also be `euclidean`,
        `matching`, `jaccard`, as well as any other distance measure
        between 1d vectors.
    df2 : pd.DataFrame
        The context embedding. 
        If df2 is not None, the distance function will be applied between the
        df-embedding and the df2-embedding.
    Raises
    ------
    ValueError
        If word is not in `df.index`.
    Returns
    -------
    pd.Series
        Ordered by closeness to `word`.
    """
    if word not in df.index:
        raise ValueError('{} is not in this VSM'.format(word))
    w = df.loc[word]
    if df2 is None:
      df2 = df
    dists = df2.apply(lambda x: distfunc(w, x), axis=1)
    return dists.sort_values()[0:15]

# -----------------------------------------------
# Here the code for a few measures of similarity 
def euclidean(u, v):
    return scipy.spatial.distance.euclidean(u, v)

def cosine(u, v):
    return scipy.spatial.distance.cosine(u, v)

# Note: the following measures should only be applied to count matrices 
def matching(u, v):
    return np.sum(np.minimum(u, v))

def jaccard(u, v):
    return 1.0 - (matching(u, v) / np.sum(np.maximum(u, v)))


#### Embedding tests

In [151]:
index = np.random.randint(0,X_df.shape[0])
word = X_df.index[index]
neighbors(word,X_df,cosine)

quotes                  0.000000
vinos                   0.557079
ripenglish              0.588347
overacting              0.602061
gomawo                  0.605229
#neverforget            0.613627
ffsss                   0.619497
bx-t                    0.620471
ingat                   0.623194
flotteste               0.631646
camel                   0.633609
#greatcomebackthough    0.636938
libbys                  0.637234
#partyrockingonidol     0.639461
i360                    0.641337
dtype: float64

In [152]:
neighbors("quotes",X_df,cosine,Y_df)

duvelkeskermis    0.601453
freemas           0.603692
hall's            0.607701
actively          0.625095
signpost          0.627758
jared             0.631085
mam's             0.631448
countri           0.631840
anlaby            0.632246
(214)             0.634015
spinch            0.634625
motivation        0.634958
inamed            0.635423
sxxt              0.643254
funko             0.644584
dtype: float64

In [154]:
index = np.random.randint(0,X_df.shape[0])
word = X_df.index[index]
neighbors(word,X_df,cosine)

mopped           0.000000
hollywoo         0.594653
#bestpartofit    0.604491
s52c             0.609266
suuucks          0.615372
vivendi          0.617818
pear             0.618398
hubert           0.618561
pheromones       0.618819
bodily           0.618852
cloves           0.623619
ultramet         0.623902
line's           0.626508
aman             0.627809
kismat           0.628252
dtype: float64

#### #*TODO*: 
- Improve preprocessing (hashtags, punctuation, ...) 
- Insert the embeddings in a classification pipeline